In [2]:
import scanpy as sc
import numpy as np
import scanpy.external as sce
import pandas as pd
import scvelo as scv
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import sys
import symphonypy as sp

# Figures aesthetics
sc.set_figure_params(dpi=150)

# Pathes
DATA_PATH = "/home/sergey/data/clonal_project"
HOME_PATH = "/home/sergey/projects/clonal_project"

# Additional functions
exec(open(f"{HOME_PATH}/tools/tools.py").read())
sns.set_style("ticks")

In [3]:
adata_counts = sc.read_h5ad(f"{DATA_PATH}/anndatas_counts/reference_E13.h5ad")

In [4]:
from tqdm import tqdm

adatas_velocity = []
for sample in tqdm(set(adata_counts.obs["sample_id"])):
    adata = sc.read_loom(f"{DATA_PATH}/cellranger/{sample}/{sample}_GEX_velocyto.loom")
    adata.obs.index = [f"{sample}:{bc[4:-1]}" for bc in adata.obs_names]
    adata.var_names_make_unique()
    adatas_velocity.append(adata)
    
adatas_velocity = adatas_velocity[0].concatenate(
    adatas_velocity[1:],
    join="outer",
    batch_key=None,
    index_unique=None,
    fill_value=0,
)[adata_counts.obs_names]

for layer in ["ambiguous", "spliced", "unspliced"]:
    adata_counts.layers[layer] = adatas_velocity[adata_counts.obs_names].layers[layer]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [10:31<00:00, 42.13s/it]


In [11]:
adata_counts.write_h5ad(f"{DATA_PATH}/anndatas_counts/reference_E13.h5ad")